# Tabular Data Formats: Comma Separated Values

In [ ]:
from src.setup import *

Delimited text files, especially comma-separated values (CSV) files, are ubiquitous.  These are text files that put multiple values on each line, and separate those values with some semi-reserved character, such as a comma.  They are almost always the exchange format used to transport data between other tabular representations, but a great deal of data both starts and ends life as CSV, perhaps never passing through other formats.  

There are a great number of deficits in CSV files, but also some notable strengths.  CSV files are the format second most susceptible to structural problems.  All formats are generally equally prone to content problems, which are not tied to the format itself.  Spreadsheets like Excel are, of course, *by a very large margin* the worst format for every kind of data integrity concern.

At the same time, delimited formats—or fixed-width text formats—are also almost the only ones you can easily open and make sense of in a text editor or easily manipulate using command-line tools for text processing.  Thereby delimited files are pretty much the only ones you can fix fully manually without specialized readers and libraries.  Of course, formats that rigorously enforce structural constraints *do avoid some* of the need to do this.  Many tabular formats **do** enforce structure and datatyping more.

One issue that you could encounter in reading CSV or other textual files is the actual character set encoding may not be the one you expect, or that is the default on your current system.  In this age of Unicode, this concern is diminishing, but only slowly, and archival files continue to exist.

### Sanity Checks

As a quick example, suppose you have just received a medium sized CSV file, and you want to make a quick sanity check on it. At this stage, we are concerned about whether the file is formatted correctly at all.  We can do this with command-line tools, even if most libraries are likely to choke on them (such as shown in the next code cell).  Of course, we could also use Python, R, or another general-purpose language if we just consider the lines as text initially.

In [ ]:
# Use try/except to avoid full traceback in example
try:
    pd.read_csv('data/big-random.csv')
except Exception as err:
    print_err(err)

What went wrong there? Let us check.

In [ ]:
# What is the general size/shape of this file?
wc = namedtuple('WordCount', 'lines words chars')
with open('data/big-random.csv') as fh:
    content = fh.read()
    lines = len(content.splitlines())
    words = len(content.split())
    chars = len(content)
wc(lines, words, chars)

Great! 100,000 rows; but there is some sort of problem on line 75 according to Pandas (and perhaps on other lines as well).  We might eyeball the file, but that could be time consuming.  Let's first check whether the lines have consistent numbers of commas on each.

In [ ]:
commas_per_line = Counter()
with open('data/big-random.csv') as fh:
    for line in fh:
        commas_per_line[line.count(',')] += 1
commas_per_line

So we have figured out already that 99,909 of the lines have the expected 5 commas.  But 46 have a deficit and 45 a surplus.  Perhaps we will simply discard the bad lines, but that is not altogether too many to consider fixing by hand, even in a text editor.  We need to make a judgement, on a per problem basis, about both the relative effort and reliability of automation of fixes versus manual approaches.  Let us take a look at a few of the problem rows.

In [ ]:
print_next = False
for lineno, line in zip(range(1, 8_500), open('data/big-random.csv')):
    if line.count(',') == 7:
        print(lineno-1, previous, end='')
        print(lineno, line, end='')
        print_next = True
    elif print_next:
        print(lineno, line, end='')
        print('--')
        print_next = False
    previous = line

<font color="darkcyan">**Note**: In the book, and in my daily work, using bash for quick exploration is often quicker than Python or R.</font>

In [ ]:
# %%bash
# grep -C1 -nP '^([^,]+,){7}' data/big-random.csv | head

Looking at these lists of Italian words and integers of slightly varying number of fields does not immediately illuminate the nature of the problem.  We likely need more domain or problem knowledge.  However, given that fewer than 1% of rows are a problem, perhaps we should simply discard them for now.  If you do decide to make a modification such as removing rows, then versioning the data, with accompanying documentation of change history and reasons, becomes crucial to good data and process provenance.

In [ ]:
lines = [l.strip().split(',') 
         for l in open('data/big-random.csv') 
         if l.count(',') == 5]
pd.DataFrame(lines)

In the code we managed, within Python, to read all rows without formatting problems.  We could also have used the `pd.read_csv()`, with `error_bad_lines=False` and some additional massaging to achieve a similar effect. Walking through it in plain Python gives you a better picture of what we exclude.

### The Good, The Bad, and The Textual Data

Let us return to some virtues and deficits of CSV files.  Here when we mention CSV, we really mean any kind of delimited file.  And specifically, text files that store tabular data nearly always use a single character for a delimiter, and end rows/records with a newline (or carriage return and newline in legacy formats).  Other than commas, probably the most common delimiters you will encounter are tabs and the pipe character `|`.  However, nearly all tools are more than happy to use an arbitrary character.

Fixed-width files are similar to delimited ones.  Technically they are different in that, although they are line oriented, they put each field of data in specific character positions within each line.  An example is used in a code cell below.  Decades ago, when Fortran and Cobol were more popular, fixed-width formats were more prevalent; my perception is that their use has diminished in favor of delimited files.  In any case, fixed-width textual data files have most of the same pitfalls and strengths as do delimited ones.

**The Bad**

Columns in delimited or flat files do not carry a data type, being simply text values.  Many tools will (optionally) make guesses about the data type, but these are subject to pitfalls.  Moreover, even where the tools accurately guess the broad type category (i.e. string vs. integer vs. real number) they cannot guess the specific bit length desired, where that matters.

Likewise, the representation used for "real" numbers is not encoded—most systems deal with IEEE-754 floating-point numbers of some length, but occasionally decimals of some specific length are more appropriate for a purpose.

The most typical way that type inference goes wrong is where the initial records in some data set have an apparent pattern, but later records deviate from this.  The software library may infer one data type but later encounter strings that cannot be cast as such.  "Earlier" and "later" here can have several different meanings.  

For many layouts, data frame libraries can guess a fixed-width format and infer column positions and data types (where it cannot guess, we could manually specify).  But the guesses about data types can go wrong.  For example, viewing the raw text, we see a fixed-width layout in `parts.fwf`.

In [ ]:
for line in open('data/parts.fwf'):
    print(line, end='')

If we deliberately only read fewer rows of the `parts.fwf` file, Pandas will infer type `int64` for the `Part_No` column.

In [ ]:
df = pd.read_fwf('data/parts.fwf', nrows=3)
df

In [ ]:
df.dtypes

However, if we read the entire file, Pandas does the "right thing" here: `Part_No` becomes a generic object, i.e. string. However, if we had a million rows instead, and the heuristics Pandas uses, for speed and memory efficiency, happened to limit inference to the first 100,000 rows, we might not be so lucky.

In [ ]:
df = pd.read_fwf('data/parts.fwf')
df

In [ ]:
df.dtypes  # type of `Part_No` changed

---

Delimited files—but not so much fixed-width files—are prone to escaping issues.  In particular, CSVs specifically often contain descriptive fields that sometimes contain commas within the value itself.  When done right, this comma should be escaped.  It is often not done right in practice.

CSV is actually a family of different dialects, mostly varying in their escaping conventions.  Sometimes spacing before or after commas is treated differently across dialects as well. One approach to escaping is to put quotes around either every string value, or every value of any kind, or perhaps only those values that contain the prohibited comma.  This varies by tool and by the version of the tool.  Of course, if you quote fields, there is potentially a need to escape those quotes; usually this is done by placing a backslash before the quote character when it is part of the value.  

An alternate approach is to place a backslash before those commas that are not intended as a delimeter but rather part of a string value (or numeric value that might be formatted, e.g. `$1,234.56`).  Guessing the variant can be a mess, and even single files are not necessarily self consistent between rows, in practice (often different tools or versions of tools have touched the data).

The corresponding danger for fixed-width files, in contrast to delimited ones, is that values become too long.  Within a certain line position range you can have any codepoints whatsoever (other than newlines).  But once the description or name that someone thought would never be longer than, say, 20 characters becomes 21 characters, the format fails.

<font color="darkcyan">A special consideration arises around reading datetime formats.  Data frame libraries that read datetime values typically have an optional switch to parse certain columns as datetime formats.  Libraries such as Pandas support heuristic guessing of datetime formats; the problem here is that applying a heuristic to each of millions of rows can be *exceedingly slow*.  Where a date format is uniform, using a manual format specifier can make it several orders of magnitude faster to read.</font>

In [ ]:
print(open('data/parts.tsv').read())

The way Pandas parses dates heuristically uses the strange USA convention of of `m/d/y`.  The option `dayfirst=True` can modify this.

In [ ]:
# Let Pandas make guesses for each row
# VERY SLOW for large tables
parts = pd.read_csv('data/parts.tsv', 
                    sep='\t', parse_dates=['Date'])
parts

We can verify that the dates are genuinely a datetime data type within the DataFrame.

In [ ]:
parts.dtypes

We have looked at some challenges and limitations of delimited and fixed-width formats, let us consider their considerable advantages as well.

**The Good**

The biggest strength of CSV files, and their delimited or fixed-width cousins, is the ubiquity of tools to read and write them.  Every library dealing with data frames or arrays, across every programming language, knows how to handle them.  Most of the time the libraries parse the quirky cases pretty well.  Every spreadsheet program imports and exports as CSV.  Every RDBMS—and most non-relational databases as well—imports and exports as CSV.  Most programmers' text editors even have facilities to make editing CSV easier.  Python has a standard library module `csv` that processes many dialects of CSV (or other delimited formats) as a line-by-line record reader.

The lack of type specification is often a strength rather than a weakness.  For example, the part numbers mentioned a few pages ago may have started out always being integers as an actual business intention, but later on a need arose to use non-integer "numbers."  With formats that have a formal type specifier, we generally have to perform a migration and copy to move old data into a new format that follows the loosened or revised constraints.

One particular case where a data type change happens especially often, in my experience, is with finite-width character fields.  Initially some field is specified as needing 5, or 15, or 100 characters for its maximum length, but then a need for a longer string is encountered later, and a fixed table structure or SQL database needs to be modified to accommodate the longer length.  Even more often—especially with databases—the requirement is underdocumented, and we wind up with a data set filled with truncated strings that are of little utility (and perhaps permanently lost data).

Text formats in general are usually flexible in this regard. Delimited files—but not fixed-width files—will happily contain fields of arbitrary length.  This is similarly true of JSON data, YAML data<sup><i>config</i></sup>, XML data, log files, and some other formats that simply utilize text, often with line-oriented records.  In all of this, data typing is very loose and only genuinely exists in the data processing steps.  That is often a great virtue.

<div id="config"
     style="display: inline-block; margin: 0 5% 0 5%; border-style: solid; border-width: 1px">
    <i>config</i><br/>
YAML usually contains relatively short configuration information rather than *data* in the prototypic sense.  TOML is a similar format in this regard, as is the older INI format.  All of these are really intended for hand editing, and hence are usually of small size, even though good APIs for reading and writing their data are common.  While you <i>could</i> put a million records into any of these formats, you will rarely or never encounter that in practice.
</div>

---

A related "looseness" of CSV and similar formats is that we often indefinitely aggregate multiple CSV files that follow the same informal schema.  Writing a different CSV file for each day, or each hour, or each month, of some ongoing data collection is very commonplace.  Many tools, such as Dask and **Spark** will seamlessly treat collections of CSV files (matching a *glob* pattern on the file system) as a single data set.  Of course, in tools that do not directly support this, manual concatenation is still not difficult.  But under the model of having a directory that contains an indefinite number of related CSV snapshots, presenting it as a single common object is helpful.